# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 19
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr19.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr19.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 540 540

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000129946,ENSG00000099866,ENSG00000099804,ENSG00000197540,ENSG00000172270,ENSG00000099822,ENSG00000099821,ENSG00000011304,ENSG00000172232,ENSG00000196415
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000129946,1.000000000,-0.01695678,0.01266588,0.07064477,-0.007171051,-0.005084999,0.011618176,-0.01330456,-0.09555396,9.432530e-02
ENSG00000099866,-0.016956775,1.00000000,0.02287341,-0.03485405,-0.084173419,0.015583024,0.133819561,0.06560198,-0.04563455,-1.706756e-02
ENSG00000099804,0.012665879,0.02287341,1.00000000,-0.02879167,-0.217811133,0.046366703,0.026759459,-0.01978384,-0.04011687,-1.151160e-01
ENSG00000197540,0.070644775,-0.03485405,-0.02879167,1.00000000,0.077361376,0.108567523,-0.005329572,0.02641652,-0.03906661,-2.154328e-02
ENSG00000172270,-0.007171051,-0.08417342,-0.21781113,0.07736138,1.000000000,-0.022704398,0.023503678,-0.06177363,-0.04705718,7.431272e-02
ENSG00000099822,-0.005084999,0.01558302,0.04636670,0.10856752,-0.022704398,1.000000000,-0.069624441,0.01501621,0.01513233,2.097045e-05


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 115

[1] -0.01136069 -0.01790573 -0.02041681 -0.02213621 -0.03013952 -0.05138622
  [7] -0.05300834 -0.05433601 -0.06422509 -0.07283311 -0.08464319 -0.09709346
 [13] -0.11830268 -0.12803437 -0.13519507 -0.13801648 -0.14350040 -0.14700723
 [19] -0.16211966 -0.17362235 -0.17702003 -0.17977390 -0.18544251 -0.19569636
 [25] -0.20384698 -0.22454652 -0.22806728 -0.23711678 -0.25297027 -0.25361613
 [31] -0.27190072 -0.27935666 -0.28843958 -0.29908039 -0.30106458 -0.33131518
 [37] -0.33548449 -0.34103411 -0.36514039 -0.37029372 -0.38270347 -0.39973722
 [43] -0.41605395 -0.41630234 -0.42509261 -0.44328782 -0.44412425 -0.45350690
 [49] -0.46348325 -0.46716801 -0.48400365 -0.49627330 -0.51546445 -0.53525153
 [55] -0.53918478 -0.54985426 -0.58883117 -0.59106695 -0.60234783 -0.61071560
 [61] -0.62496346 -0.62911801 -0.65731031 -0.67914787 -0.67921098 -0.69905699
 [67] -0.71003053 -0.76051269 -0.76131008 -0.78682092 -0.78907564 -0.80589853
 [73] -0.80925632 -0.83365618 -0.85533747 -0.87623916 -0.90081565 -0.90523324
 [79] -0.91488880 -0.92099044 -0.93481182 -0.99767172 -1.03515416 -1.06092579
 [85] -1.07910565 -1.09857625 -1.11063625 -1.12318607 -1.17004260 -1.18143538
 [91] -1.26360350 -1.26841751 -1.30526960 -1.35471671 -1.39773668 -1.41585873
 [97] -1.45518660 -1.50450803 -1.55253817 -1.57455308 -1.64455399 -1.70818171
[103] -1.74612211 -1.78521993 -1.80017133 -1.90032832 -1.98628312 -2.00781397
[109] -2.15532986 -2.43728618 -2.60489237 -2.80907435 -3.17748310 -4.28457545
[115] -5.48397845

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 540 540

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000129946,ENSG00000099866,ENSG00000099804,ENSG00000197540,ENSG00000172270,ENSG00000099822,ENSG00000099821,ENSG00000011304,ENSG00000172232,ENSG00000196415
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000129946,1.000000000,-0.01695678,0.01266588,0.07064477,-0.007171051,-0.005084999,0.011618176,-0.01330456,-0.09555396,9.432530e-02
ENSG00000099866,-0.016956775,1.00000000,0.02287341,-0.03485405,-0.084173419,0.015583024,0.133819561,0.06560198,-0.04563455,-1.706756e-02
ENSG00000099804,0.012665879,0.02287341,1.00000000,-0.02879167,-0.217811133,0.046366703,0.026759459,-0.01978384,-0.04011687,-1.151160e-01
ENSG00000197540,0.070644775,-0.03485405,-0.02879167,1.00000000,0.077361376,0.108567523,-0.005329572,0.02641652,-0.03906661,-2.154328e-02
ENSG00000172270,-0.007171051,-0.08417342,-0.21781113,0.07736138,1.000000000,-0.022704398,0.023503678,-0.06177363,-0.04705718,7.431272e-02
ENSG00000099822,-0.005084999,0.01558302,0.04636670,0.10856752,-0.022704398,1.000000000,-0.069624441,0.01501621,0.01513233,2.097045e-05


,ENSG00000129946,ENSG00000099866,ENSG00000099804,ENSG00000197540,ENSG00000172270,ENSG00000099822,ENSG00000099821,ENSG00000011304,ENSG00000172232,ENSG00000196415
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000129946,1.000000000,2.547019e-02,-0.002709085,3.277208e-02,0.002030447,0.008470579,-0.006152145,-0.014888669,-0.043178442,0.0583894298
ENSG00000099866,0.025470192,1.000000e+00,0.012474164,5.611906e-05,-0.027287306,-0.005948942,0.040720592,0.048897984,0.015820495,-0.0385978192
ENSG00000099804,-0.002709085,1.247416e-02,1.000000000,-2.346554e-02,-0.101258284,0.008752329,0.022641420,-0.001210648,-0.013212328,-0.0570643847
ENSG00000197540,0.032772083,5.611906e-05,-0.023465544,1.000000e+00,0.060788697,0.054300357,-0.027272314,0.005990444,-0.016616806,-0.0019063233
ENSG00000172270,0.002030447,-2.728731e-02,-0.101258284,6.078870e-02,1.000000000,0.014532654,-0.012761344,-0.069060513,-0.006831454,0.0319498193
ENSG00000099822,0.008470579,-5.948942e-03,0.008752329,5.430036e-02,0.014532654,1.000000000,-0.044776900,0.021578927,0.021745899,-0.0001460568


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)